In [1]:
import numpy as np
import pickle
import random
import json
import os
import plotly.graph_objects as go
import sklearn.cluster as cluster

from tqdm.notebook import tqdm

# A Process class which emulates a process in a server. It has a pid and a length measured in numbers of instructions.
# Furthermore, it possesses the __repr__ method which is used to print the object.
# Moreover, it possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Process:
    def __init__(self, pid, length):
        self.pid = pid
        self.length = length

    def __repr__(self):
        return f"Process(pid={self.pid}, length={self.length})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

In [2]:
# A Server class which emulates a server. It has a name, a number of cpus, the speed of its cpu measured in GHz and a workload list, which represents 
# the processes that are assigned to the server. The workload starts as empty. 
# Furthermore, it possesses the __repr__ method which is used to print the object.
# Moreover, it possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Server:
    def __init__(self, name, cpus, cpu_speed):
        self.name = name
        self.cpus = cpus
        self.cpu_speed = cpu_speed

    def __repr__(self):
        return f"Server(name={self.name}, cpus={self.cpus}, cpu_speed={self.cpu_speed}"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)


In [3]:

# A Solution class, wich represents a solution to the problem. is made of a list of tuples in the form <server, associated processes>
# The solution object possesses a fitness value, which is a vector of the fitness of each servers with respect to their associated processes. 
# The fitness is calculated as the sum of the length of the processes assigned to the server divided by the number of cpus of the server times their speed.
# It possesses the __repr__ method which is used to print the object.
# It possesses a to_json method which is used to convert the object to a json string and a from_json to convert it back.

class Solution:
    def __init__(self, solution):
        self.solution = solution
        self.fitness = self.calculate_fitness()

    def __repr__(self):
        return f"Solution(solution={self.solution}, fitness={self.fitness})"

    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string):
        return json.loads(json_string)

    def calculate_fitness(self):
        fitness = []
        for server, processes in self.solution:
            fitness.append(sum([process.length for process in processes]) / (server.cpus * server.cpu_speed))
        return fitness

In [4]:
# A clusterization algorithm which:
# 1. Clusterizes the processes in N subsets, with N equal to the number of servers
# 2. Assign the servers to a cluster in a greedy fashion.
#    > The server that fits best the cluster is assigned to it.
#    > The server that fits best the cluster is the one that has the highest fitness value.
#    > The fitness value is calculated as the sum of the length of the processes assigned to the server times by the number of cpus of the server times their speed.

class GreeedyClusterization:
    def __init__(self, processes, servers):
        self.processes = processes
        self.servers = servers
        self.solution = self.clusterize()

    def clusterize(self):
        clusters = []
        for i in range(len(self.servers)):
            clusters.append([])
        for i in range(len(self.processes)):
            clusters[i % len(self.servers)].append(self.processes[i])
        solution = []
        for i in range(len(self.servers)):
            solution.append((self.servers[i], clusters[i]))
        return Solution(solution)

        

In [ ]:
# Similar to before but with the kmeans algorithm

class KmeansClusterization:
    def __init__(self, processes, servers):
        self.processes = processes
        self.servers = servers
        self.solution = self.clusterize()

    def clusterize(self):
        processes = [process.length for process in self.processes]
        servers = [(server.cpus, server.cpu_speed) for server in self.servers]
        kmeans = cluster.KMeans(n_clusters=len(self.servers), random_state=0).fit(servers)
        clusters = [[] for i in range(len(self.servers))]
        for i in range(len(processes)):
            clusters[kmeans.labels_[i]].append(self.processes[i])
        solution = []
        for i in range(len(self.servers)):
            solution.append((self.servers[i], clusters[i]))
        return Solution(solution)